In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# __str__, __repr__
* 인스턴스의 문자열 표현식 변경

In [7]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        # 'Pair(%r, %r)' % (self.x, slef.y) 와 같다 
        return 'Pair({0.x!r}, {0.y!r})'.format(self) 
    def __str__(self):
        return 'Pair({0.x!s}, {0.y!s})'.format(self)
    

repr : 인스턴스의 코드 표현식을 반환 일반적으로 인스턴스를 재생성할때 입력하는 텍스트이다. 

In [3]:
p = Pair(3,4)
p
print(p)

Pair(3, 4)

Pair(3, 4)


In [4]:
print('p is {0!r}'.format(p))

p is Pair(3, 4)


In [5]:
print('p is {0}'.format(p))

p is Pair(3, 4)


ㅡr! 는 기본값으로 __str__ 대신 __repr__ 을 사용함을 가리킨다. 

str 을 정의하지 않으면 repr 을 대신 사용한다. 

# format
* 문자열 서식화 조절 

In [8]:
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
    }

In [9]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [10]:
d = Date(2011, 11, 11)
format(d)

'2011-11-11'

In [11]:
format(d, 'mdy')

'11/11/2011'

In [12]:
'The date is {:ymd}'.format(d)

'The date is 2011-11-11'

# with, enter, exit
* 객체 콘텍스트 관리 프로토콜 지원 

In [13]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        #self.sock=None
        self.connections = []

    def __enter__(self):
#         if self.sock is not None:
#             raise RuntimeError('Already connected')
#         self.sock = socket(self.family, self.type)
#         self.sock.connect(self.address)
#         return self.sock
    
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connections.append(sock)
        return sock
                
    def __exit__(self, exc_ty, exc_val, tb):
#         self.sock.close()
#         self.sock = None
        self.connections.pop().close()

In [14]:
from functools import partial

conn = LazyConnection(('www.python.org', 80))

conn = LazyConnection(('www.python.org', 80))
with conn as s:
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))

print('Got %d bytes' % len(resp))


26

22

2

Got 392 bytes


처음으로 with 를 만나면 enter() 메소드가 호출된다. enter() 의 반환값이 있다면 as 로 나타낸 변수에 위치시킨다. 그후 with 내부를 실행하고 마지막으로 exit() 으로 소거작업을 한다. 

예외가 발생해도 exit는 실행된다.
exit 메소드의 세가지 인자에 예외타입, 값, 트레이스백(traceback) 이 있다. 

In [17]:

with conn as s1, conn as s2:
    s1.send(b'GET /downloads HTTP/1.0\r\n')
    s2.send(b'GET /index.html HTTP/1.0\r\n')
    s1.send(b'Host: www.python.org\r\n')
    s2.send(b'Host: www.python.org\r\n')
    s1.send(b'\r\n')
    s2.send(b'\r\n')
    resp1 = b''.join(iter(partial(s1.recv, 8192), b''))
    resp2 = b''.join(iter(partial(s2.recv, 8192), b''))

print('resp1 got %d bytes' % len(resp1))
print('resp2 got %d bytes' % len(resp2))

25

26

22

22

2

2

resp1 got 391 bytes
resp2 got 392 bytes
